# Matrix-model Basic Example
This example aims to highlight the follow:
* build a complete matrix of distances/times between a collection of locations

## Requirements
This example assumes you've configured an api-key with the required services enabled (see the [portal](portal.icepack.ai) for configuration details) and that you're familiar with loading and working with protobuf models.

## Data
We're going to load a small sample data set (just 5 locations) which will yeild 20 distance pairs.


In [ ]:
import pandas
df = pandas.read_csv('../sample_data/publist.csv').head(5)
print(df.head())

## Model helper
There are several common code-snippets which are useful when working with the icepack models. We use the same standard for post/get queries for all models. We're going to use the api helper in this context to help manage the calls to the api.

We're going to:
1. load the helper
2. instantiate a matrix model request
3. populate the request
4. post the request (helper class)
5. get the response (helper class)

### Build the request

In [ ]:
exec(open('apiHelper.py').read()) # import some api-helper classes which we've written for you.

api = apiHelper(modelType="matrix-vyv95n7wchpl") # set the model type to the matrix model

mr = matrix_vyv95n7wchpl_pb2.MatrixRequest()

for index, row in df.iterrows():
    l = matrix_vyv95n7wchpl_pb2.Location()
    l.id = row.id
    l.geocode.longitude = row.X
    l.geocode.latitude = row.Y
    mr.locations.append(l)                    # add the location definition to the list of locations
    mr.sources.append(row.id)                 # add the id to the list of sources

mr.distanceUnit = 0
mr.durationUnit = 1
print(mr)


So this model has 5 locations and 5 sources. In the world of the matrix model this means that we want a complete matrix between these 5 locations (because the destinations have been omitted). 

### Send the request to the api
Because we're using a common interface we can use the apihelper Post and Get methods to send and retrieve the result

In [ ]:
reqId = api.Post(mr)  # send the request
print(reqId) # if you want to see what the guid looks like.
sol = api.Get(reqId)  # get the response


In [ ]:
print(len(sol.elements)) # the number of elements returned by the api

In [ ]:
print(sol.elements[1:3]) # the first two elements, we can see the distance and duration.

### Some notes
You'll see we get 20 elements in the response from the api - which is 5*(5-1) (because the diagonal is always zero.)

We can tabulate the response by calling the `tabulate` method we've bundled here.

In [ ]:
matrix = tabulate(mr, sol)
matrix


This matrix is expressed in long-form. Nothing wrong with that, but you might prefer to see it in wide form. Pandas has a nice function for this.


In [ ]:
import numpy as np
dm = pandas.crosstab(index= matrix['fromId'], 
                columns=matrix['toId'], 
                values = matrix['distance'], aggfunc = np.sum).fillna(0)

# this creates a distance matrix 
dm

In [ ]:
tm = pandas.crosstab(index= matrix['fromId'], 
                columns=matrix['toId'], 
                values = matrix['duration'], aggfunc = np.sum).fillna(0)
#and this creates a time matrix
tm

### What next?
Now that you've mastered this basic example - lets try something a little trickier (see the intermediate example next).